In [2]:
import networkx as nx
import wikipediaapi
#import graphviz

In [3]:
user_agent = 'wikipedia-graph (danielwarkus@hotmail.com)'

In [4]:
wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='de')

In [5]:
python_page = wiki.page('Python (Programmiersprache)')

In [6]:
def get_backlinks(page):
    backlinks = []
    for link in page.backlinks.values():
        backlinks.append(link.title)
    return backlinks

In [7]:
def get_links(page):
    links = []
    for link in page.links.values():
        links.append(link.title)
    return links

In [8]:
def get_page(link):
    return wiki.page(link)

In [9]:
def make_graph(page, depth):
    graph = nx.DiGraph()
    graph.add_node(page.title) # weg damit
    links = get_links(page)
    for link in links:
        graph.add_edge(page.title, link)
        if depth > 0:
            next_page = get_page(link)
            graph = nx.compose(graph, make_graph(next_page, depth-1))
    return graph

Optimieren! Liste von Knoten -> Subgraph

In [13]:
def make_graph_optimized(page):
    graph = nx.DiGraph()
    page_links = get_links(page)
    
    for link in page_links:
        graph.add_edge(page.title, link)
        
        new_page = get_page(link)
        new_page_links = get_links(new_page)
        for link in new_page_links:
            graph.add_edge(new_page.title, link)
    return graph

In [9]:
def make_backgraph(page, depth):
    backgraph = nx.DiGraph()
    backgraph.add_node(page.title)
    backlinks = get_backlinks(page)
    for link in backlinks:
        backgraph.add_edge(page.title, link)
        if depth > 0:
            next_page = get_page(link)
            backgraph = nx.compose(backgraph, get_graph(next_page, depth-1))
    return backgraph

# Erstelle den Ursprungsgraphen
### Hier wird ein Graph erstellt, der alle Links (und deren Links) von der Bienen-Seite enthält.
### Die Artikelnamen sind dabei die Knoten und die Verlinkungen die Kanten. 

In [14]:
graph = make_graph(python_page, 1)

In [15]:
graph_optimized = make_graph_optimized(python_page)

In [11]:
backlinks = get_backlinks(python_page)

In [12]:
print(backlinks)

['Ada (Programmiersprache)', 'Diskussion:BASIC', 'C (Programmiersprache)', 'C++', 'Garbage Collection', 'JavaScript', 'Julianisches Datum', 'Kommandozeileninterpreter', 'Diskussion:Kommandozeileninterpreter', 'Lisp', 'Linux', 'LaTeX', 'Monty Python', 'Message-Digest Algorithm 5', 'Makroökonomie', 'Apache OpenOffice', 'Zeittafel der Programmiersprachen', 'Java (Programmiersprache)', 'Perl (Programmiersprache)', 'Quelltext', 'Regulärer Ausdruck', 'Rekursion', 'Scalable Vector Graphics', 'Sprache', 'Smalltalk (Programmiersprache)', 'Skriptsprache', 'Diskussion:Skriptsprache', 'Tcl', 'World Wide Web', 'Extensible Markup Language', 'XSL Transformation', 'Zwischenablage', 'Zeichenkette', 'At-Zeichen', 'Python', 'Tk (Toolkit)', 'Gnome', 'CAD', 'Dijkstra-Algorithmus', 'Potenz (Mathematik)', 'Freie Software', 'Interpreter', 'Präsentationsprogramm', 'Vollständiger Graph', 'Isomorphie von Graphen', 'Qt (Bibliothek)', 'KDevelop', 'Diskussion:Ebolafieber', 'GTK (Programmbibliothek)', 'Make', 'Stape

In [13]:
backgraph = get_backgraph(python_page, 1)

In [14]:
complete_graph = nx.compose(graph, backgraph)
# ANMERKUNG: ZUM TEST WURDE COMPLETE_GRAPH VERWENDET;; SONST NUR GRAPH

In [15]:
# from graph, export edges as a tuple to a csv file
edges = graph.edges
with open('edges_original.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuchserklärung
#### Ziel ist es, Artikelvorschläge zu generieren, die thematisch zu Bienen passen. Dabei werde ich 4 verschiedene Ansätze testen (dabei wird immer der höchste Grad der Knoten betrachtet):
#### 1. Extrahiere die Knoten mit dem höchsten Grad (ungefiltert)
#### 2. Betrachte nur die direkten Nachfolger von Bienen
#### 3. Betrachte nur die Knoten, die zu Bienen verlinkt sind
#### 4. Betrachte nur die direkten Nachfolger von Bienen, die auch eine Verbindung zurück zu Bienen haben (beide Bedingungen)

#### Die Tests 2-4 wurde ich sowohl am Hauptgraphen, als auch an einem Subgraphen durchführen, um zu schauen ob dies ein Unterschied macht.

# Versuch 1: Extrahiere die Knoten mit dem höchsten Grad
#### Im ersten Versuch werden die Knoten mit dem höchsten Grad extrahiert. Man kann erkennen, dass die Artikelvorschläge zu oberflächlich sind. 

In [16]:
# print the top 3 nodes with the highest degree
top_3_nodes = sorted(complete_graph.degree, key=lambda x: x[1], reverse=True)[:3]
print(top_3_nodes)

[('Benutzer:Dapete/Datenbankabfragen/Verwaiste Artikeldiskussionen', 40045), ('Wikipedia:Einbänder/Stichworte', 19370), ('Wikipedia:Artikel, die es in allen Wikipedias geben sollte/Erweitert', 9619)]


# Versuch 2: Extrahiere die direkten Nachfolger von Bienen
#### Um die Auswahl passender Artikel einzuschränken, werden nur die direkten Nachfolger von Bienen betrachtet.
#### Diese neue Bedingung soll dazu führen, nur Artikel in Betracht zu ziehen, die auch von der Biene refenziert werden.

#### Getestet habe ich diese Bedinung an zwei Graphen:
#### 1. Der ursprüngliche Graph
#### 2. Ein Subgraph, der nur die Knoten enthält, die von Bienen referenziert werden.
 

In [17]:
links_python = get_links(python_page)

In [18]:
reduced_node_list = [node for node in complete_graph.nodes() if node in links_python]

## Erstelle Subgraphen und extrahiere die Knoten mit dem höchsten Grad
#### Hier wird ein Subgraph erstellt, der nur die Knoten enthält, die direkte Nachfolger von Bienen sind.
#### Die Ergebnisse sind besser, als die des ursprünglichen Graphen.

In [19]:
# create a subgraph
subgraph = complete_graph.subgraph(reduced_node_list)

In [20]:
top_3_reduced_nodes = sorted(subgraph.degree(), key=lambda x: x[1], reverse=True)[:3]
print(top_3_reduced_nodes)

[('Java (Programmiersprache)', 113), ('C++', 109), ('Betriebssystem', 104)]


## Extrahiere Knoten mit dem höchsten Grad aus Hauptgraphen
#### Hier zeigt die Zusatzbedingung keine Verbesserung.

In [21]:
top_3_reduced_nodes = sorted(complete_graph.degree(reduced_node_list), key=lambda x: x[1], reverse=True)[:3]
print(top_3_reduced_nodes)

[('Wikipedia:Lesenswerte Artikel', 4448), ('Betriebssystem', 888), ('Java (Programmiersprache)', 791)]


## Zusammenfassung Versuch 2
#### Man kann sehen, dass die reine Bedingung an den Knoten des Hauptgraphen noch nicht aussreicht für gute Ergebnisse. -> Zu oberflächliche Themen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.  

In [22]:
# export the subgraph to a csv file
edges = subgraph.edges
with open('edges_reduced.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuch 3: Extrahiere Knoten, die zu Bienen verlinkt sind
#### Hier wird die Bedingung getestet, dass nur die Knoten betrachtet werden die zu Bienen verlinkt sind.

#### Ebenfalls werde ich dies hier am Hauptgraphen und an einem Subgraphen testen.

In [23]:
nodes_with_edge_to_python = [node for node in complete_graph.nodes() if complete_graph.has_edge(node, 'Python (Programmiersprache)')]

## Mit Ursprungsgaphen
#### Es zeigt sich, dass wir ein durchwegs gute Themenvorschläge erhalten.

In [24]:
top_3_linked = sorted(complete_graph.degree(nodes_with_edge_to_python), key=lambda x: x[1], reverse=True)[:3]
print(top_3_linked)

[('Benutzer:Dapete/Datenbankabfragen/Verwaiste Artikeldiskussionen', 40045), ('Wikipedia:Einbänder/Stichworte', 19370), ('Wikipedia:Artikel, die es in allen Wikipedias geben sollte/Erweitert', 9619)]


## Mit Subgraphen
#### Deutlich erkennt man, dass die Themenvorschläge zu spezifisch sind.

In [25]:
subgraph_linked = graph.subgraph(nodes_with_edge_to_python)
top_3_linked = sorted(subgraph_linked.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_3_linked)

[('Java (Programmiersprache)', 98), ('C++', 95), ('Perl (Programmiersprache)', 79), ('C (Programmiersprache)', 76), ('Ruby (Programmiersprache)', 73)]


In [26]:
# export the subgraph to a csv file
edges = subgraph_linked.edges
with open('edges_linked.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 3
#### Diese Bedingung sorgt für signifikant bessere Vorschläge am Hauptgraphen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.

# Versuch 4: Extrahiere Knoten mit beiden Bedingungen
#### Zum Schluss setzten wir beide Bedingungen vorraus. Auch hier wird dies am Hauptgraphen und an einem Subgraphen getestet.

In [27]:
nodes_with_edge_to_bienen_reduced = [node for node in reduced_node_list if complete_graph.has_edge(node, 'Python (Programmiersprache)')]
print(nodes_with_edge_to_bienen_reduced)

['C++', 'C (Programmiersprache)', 'Garbage Collection', 'Java (Programmiersprache)', 'Just-in-time-Kompilierung', 'Kommandozeile', 'MacOS', 'Plattformunabhängigkeit', 'Reflexion (Programmierung)', 'Skriptsprache', 'Zeichenkette', 'Wikipedia:Lesenswerte Artikel', 'ABC (Programmiersprache)', 'Centrum Wiskunde & Informatica', 'Guido van Rossum', 'Höhere Programmiersprache', 'Interpreter', 'Strukturierte Programmierung', 'Typisierung (Informatik)', 'Funktionale Programmierung', 'Allzweck-Programmiersprache', 'Kommandozeileninterpreter', 'Vim', 'Amoeba (Betriebssystem)', 'Anaconda (Python-Distribution)', 'Conda (Paketverwaltung)', 'IPython', 'Project Jupyter', 'Spyder (Software)', 'Apache OpenOffice', 'Relationale Datenbank', 'Ausnahmebehandlung', 'Objektorientierte Programmierung', 'Vererbung (Programmierung)', 'Haskell (Programmiersprache)', 'Perl (Programmiersprache)', 'Tcl', 'Smalltalk (Programmiersprache)', 'Blender (Software)', 'YouTube', 'Einrückungsstil', 'Boost (C++-Bibliothek)', '

## Extraktion mit beiden Bedingungen am Hauptgraphen
#### Hier wird die Kombination von beiden Bedingungen am Hauptgraphen getestet. Diese Erbnisse unterschieden sich jedoch nicht von den Ergebnissen des dritten Versuchs.

In [28]:
top_3_clean = sorted(complete_graph.degree(nodes_with_edge_to_bienen_reduced), key=lambda x: x[1], reverse=True)[:3]
print(top_3_clean)

[('Wikipedia:Lesenswerte Artikel', 4448), ('Java (Programmiersprache)', 791), ('C++', 727)]


## Erstellung und Extraktion mit beiden Bedingungen an eines Subgraphen
#### Dieser Subgraph enthält nur die Knoten, die beide Bedingungen.
#### Hier sind die ebenso wie in Versuch 3 die Themenvorschläge identisch. Das Einzige worin sich die Ergebnisse unterscheiden, ist eine abnahme im Degree.

In [29]:
subgraph_clean = complete_graph.subgraph(nodes_with_edge_to_bienen_reduced)

In [30]:
top_3_clean = sorted(subgraph_clean.degree(nodes_with_edge_to_bienen_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_3_clean)

[('C++', 74), ('Java (Programmiersprache)', 72), ('C (Programmiersprache)', 62), ('Ruby (Programmiersprache)', 55), ('Perl (Programmiersprache)', 54)]


In [31]:
# export the subgraph to a csv file
edges = subgraph_clean.edges
with open('edges_predecessors.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 4:
#### Auch hier schneidet der Hauptgraph besser ab als der Subgraph.    

# Fazit
#### Die besten Ergebnissen wurden am Hauptgraphen mit der Bedingung erzielt, dass nur die Knoten betrachtet werden, die zu Bienen verlinkt sind. Dass beide Bedungungen erfüllt sein müssen, hat keinen Unterschied gemacht.
#### Der Subgraph hat bei allen Versuchen zu spezifische Artikel vorgeschlagen. Dies könnte man damit erklären, dass Informationen vom Hauptgraphen verloren gehen, da nur ein Teil der Knoten betrachtet wird. Dadurch fehlen Ingoing und Outgoing Links, die den Degree beeinflussen.   

In [32]:
backlinks = python_page.backlinks


In [33]:
top_x_backlinks = sorted(backlinks, key=lambda x: x[1], reverse=True)[:5]
print(top_x_backlinks)

['O’Reilly Open Source Convention', 'Kürzester Pfad', 'Führende Null und Füllnull', 'Höhere Programmiersprache', 'Nächste-Nachbarn-Klassifikation']


In [34]:
pr_result = nx.pagerank(complete_graph)

In [35]:
top_n_pr = sorted(pr_result, key=lambda x: pr_result[x], reverse=True)[:5]

In [36]:
print(top_n_pr)

['Python (Programmiersprache)', 'Programmiersprache', 'Betriebssystem', 'Version (Software)', 'Softwareentwickler']


In [37]:
filtered_pr = nx.pagerank(graph)
top_n_filtered_pr = sorted(filtered_pr, key=lambda x: filtered_pr[x], reverse=True)[:5]
print(top_n_filtered_pr)

['Programmiersprache', 'Python (Programmiersprache)', 'Betriebssystem', 'Version (Software)', 'C (Programmiersprache)']
